In [1]:
# Oxidation State Calculator with Interactive Widgets
# This script creates a user-friendly interface for calculating oxidation states 
# of chemical compounds, both for single formulas and Excel files with multiple compounds

import pandas as pd
import numpy as np

import ipywidgets as widgets
from IPython.display import display

import plotly.graph_objects as go

import io
from approximate import ApprOXimate

In [2]:
class OxidationStateCalculator:
    """
    A complete oxidation state calculator with interactive user interface.
    This class handles single formula analysis, batch processing of Excel files,
    and interactive charge balance plotting with a tabbed interface.
    """
    
    def __init__(self, analyzer):
        """
        Initialize the calculator with an ApprOXimate analyzer instance
        
        Args:
            analyzer: ApprOXimate analyzer instance for processing chemical formulas
        """
        self.analyzer = analyzer
        
        # Store processed results for user access
        self.processed_df = None
        self.single_result_df = None
        
        # Initialize the user interface
        self._create_widgets()
        self._setup_event_handlers()
        self.interface = self._create_tabbed_layout()
    
    # ================================
    # CORE PROCESSING METHODS
    # ================================
    
    def process_single_formula(self, input_text):
        """
        Process a single chemical formula and return oxidation state data
        
        Args:
            input_text (str): Chemical formula (e.g., "H2SO4")
        
        Returns:
            tuple: (DataFrame with results, summary message)
        """
        try:
            # Parse the chemical formula using the analyzer
            parsed_formula = self.analyzer.parse_formula(input_text)
            if parsed_formula is None:
                return None, "Error: Unable to parse formula"
            
            # Calculate charge balance and oxidation states
            calculation_result = self.analyzer.charge_balance(parsed_formula)
            result_parts = calculation_result.split(";")
            
            # Prepare data for creating a table (DataFrame)
            element_data = []
            final_charge = None
            
            # Extract information from each part of the result
            for item in result_parts:
                if item.startswith("FinalChargeBalance:"):
                    # Extract the final charge balance value
                    charge_label, charge_value = item.split(":")
                    final_charge = float(charge_value)
                else:
                    # Extract element, oxidation state, and quantity
                    element, oxidation_state, quantity = item.split(":")
                    element_data.append({
                        'Element': element,
                        'Oxidation_State': int(float(oxidation_state)),
                        'Quantity': float(quantity)
                    })
            
            # Create a table (DataFrame) from the extracted data
            results_table = pd.DataFrame(element_data)
            summary_message = f"Final Charge Balance: {final_charge}"
            
            return results_table, summary_message
            
        except Exception as error:
            return None, f"Error processing formula: {str(error)}"
    
    def extract_oxidation_data(self, result_string):
        """
        Parse the analyzer result string and extract oxidation state information
        
        Args:
            result_string (str): Raw result from the analyzer
        
        Returns:
            dict: Dictionary with element oxidation states and quantities
        """
        result_parts = result_string.split(";")
        element_data = {}
        
        for item in result_parts:
            # Skip the final charge balance entry
            if not item.startswith("FinalChargeBalance:"):
                element, oxidation_state, quantity = item.split(":")
                oxidation_state = int(float(oxidation_state))
                quantity = float(quantity)
                
                # Initialize element dictionary if not exists
                if element not in element_data:
                    element_data[element] = {}
                
                # Add or sum quantities for the same oxidation state
                if oxidation_state in element_data[element]:
                    element_data[element][oxidation_state] += quantity
                else:
                    element_data[element][oxidation_state] = quantity
        
        return element_data
    
    def find_all_oxidation_combinations(self, df, composition_column):
        """
        Scan all compositions to find which element-oxidation state combinations exist
        This helps create appropriate columns in the final table
        
        Args:
            df (DataFrame): Input data containing compositions
            composition_column (str): Name of column with chemical formulas
        
        Returns:
            list: Sorted list of element-oxidation state combinations (e.g., ["H+1", "O-2"])
        """
        found_combinations = set()
        
        # Check each composition in the dataset
        for index, row in df.iterrows():
            composition = str(row[composition_column])
            try:
                parsed_formula = self.analyzer.parse_formula(composition)
                if parsed_formula is not None:
                    result = self.analyzer.charge_balance(parsed_formula)
                    element_data = self.extract_oxidation_data(result)
                    
                    # Add each element-oxidation state combination to our set
                    for element, oxidation_states in element_data.items():
                        for oxidation_state in oxidation_states.keys():
                            # Format: element name + oxidation state with sign
                            combination = f"{element}{oxidation_state:+d}"
                            found_combinations.add(combination)
            except:
                # Skip problematic entries and continue
                continue
        
        return sorted(list(found_combinations))
    
    def process_excel_file(self, file_content, composition_column):
        """
        Process an Excel file containing multiple chemical compositions
        
        Args:
            file_content (bytes): Content of the uploaded Excel file
            composition_column (str): Name of column containing chemical formulas
        
        Returns:
            tuple: (Processed DataFrame, status message)
        """
        try:
            # Read the Excel file into a DataFrame (table)
            input_df = pd.read_excel(io.BytesIO(file_content))
            
            # Check if the specified column exists
            if composition_column not in input_df.columns:
                return None, f"Column '{composition_column}' not found in Excel file"
            
            # Find all element-oxidation state combinations in the dataset
            oxidation_combinations = self.find_all_oxidation_combinations(input_df, composition_column)
            
            # Create a copy of the original data to add new columns
            result_df = input_df.copy()
            
            # Add a column for each element-oxidation state combination
            for combination in oxidation_combinations:
                result_df[combination] = np.nan  # Initialize with empty values
            
            # Add column for final charge balance
            result_df['Final_Charge_Balance'] = np.nan
            
            # Process each row (composition) in the dataset
            for index, row in input_df.iterrows():
                composition = str(row[composition_column])
                
                try:
                    # Parse and analyze the composition
                    parsed_formula = self.analyzer.parse_formula(composition)
                    if parsed_formula is not None:
                        result = self.analyzer.charge_balance(parsed_formula)
                        result_parts = result.split(";")
                        
                        # Extract final charge balance
                        final_charge = None
                        for item in result_parts:
                            if item.startswith("FinalChargeBalance:"):
                                charge_label, charge_value = item.split(":")
                                final_charge = float(charge_value)
                                break
                        
                        # Store the final charge balance
                        result_df.at[index, 'Final_Charge_Balance'] = final_charge
                        
                        # Get oxidation state data for this composition
                        element_data = self.extract_oxidation_data(result)
                        
                        # Initialize all relevant columns to 0 for elements in this composition
                        elements_in_composition = set(element_data.keys())
                        for element in elements_in_composition:
                            for combination in oxidation_combinations:
                                if combination.startswith(element):
                                    result_df.at[index, combination] = 0
                        
                        # Fill in the actual calculated values
                        for element, oxidation_states in element_data.items():
                            for oxidation_state, quantity in oxidation_states.items():
                                column_name = f"{element}{oxidation_state:+d}"
                                if column_name in oxidation_combinations:
                                    result_df.at[index, column_name] = quantity
                                    
                except Exception as error:
                    print(f"Error processing composition '{composition}' at row {index}: {str(error)}")
                    continue
            
            return result_df, "Success"
            
        except Exception as error:
            return None, f"Error processing Excel file: {str(error)}"
    
    # ================================
    # INTERACTIVE PLOTTING METHODS
    # ================================
    
    def compute_charge_balances(self, parsed_formula, alkali_metal, interval, max_range):
        """Compute charge balances across a range of alkali metal values."""
        alkali_metal_amount = parsed_formula.get(alkali_metal, 0)
        if alkali_metal_amount == 0:
            return [], [], f"Error: {alkali_metal} not found in formula"

        values = np.arange(interval, min(alkali_metal_amount, max_range), interval)
        charge_balances = []
        
        for val in values:
            try:
                modified_formula = {**parsed_formula, alkali_metal: val}
                result = self.analyzer.charge_balance(modified_formula)
                charge_balance = float(result.split(';')[-1].split(':')[-1])
                charge_balances.append(charge_balance)
            except Exception:
                charge_balances.append(np.nan)
        
        return values, charge_balances, "Success"

    def create_charge_balance_plot(self, values, charge_balances, alkali_metal, formula):
        """Creates a plotly graph with fixed layout and visible legend."""
        fig = go.Figure()

        # Main trace: Charge Balance
        fig.add_trace(go.Scatter(
            x=values,
            y=charge_balances,
            mode='lines+markers',
            name=f'{alkali_metal} Charge Balance',
            line=dict(width=2),
            marker=dict(size=4)
        ))

        # Manual horizontal line at y = 0
        if len(values) > 0:
            fig.add_trace(go.Scatter(
                x=[min(values), max(values)],
                y=[0, 0],
                mode='lines',
                name='Charge Neutral Line',
                line=dict(color='red', dash='dash'),
                showlegend=True
            ))

            fig.update_layout(
                xaxis=dict(
                    title=f'Amount of {alkali_metal}',
                    fixedrange=True,
                    tickformat=".2f",
                ),
                yaxis=dict(
                    title='Charge Balance',
                    fixedrange=True,
                    tickformat=".3f"
                ),
                template='plotly_white',
                hovermode='x unified',
                width=700,
                height=500,
                margin=dict(l=60, r=20, t=60, b=60),
                legend=dict(
                    orientation='h',
                    yanchor='bottom',
                    y=1.02,
                    xanchor='right',
                    x=1
                )
            )

        return fig
    
    # ================================
    # USER INTERFACE CREATION METHODS
    # ================================
    
    def _create_widgets(self):
        """Create all the user interface widgets"""
        
        # ----------------
        # Single Formula Section Widgets
        # ----------------
        
        # Text input box for entering chemical formulas
        self.formula_input = widgets.Text(
            value='',
            description='Chemical Formula:',
            placeholder='Enter formula (e.g., H2SO4)',
            disabled=False,
            style={'description_width': '140px'}
        )
        
        # Button to trigger calculation
        self.calculate_button = widgets.Button(
            description='Calculate Oxidation States',
            button_style='info',  # Blue color
            tooltip='Click to analyze the chemical formula',
            icon='check',
            layout=widgets.Layout(width='250px', height='40px')
        )
        
        # Area to display results for single formula
        self.single_results_display = widgets.HTML(value="",)
        
        # ----------------
        # Excel Processing Section Widgets
        # ----------------
        
        # File upload widget for Excel files
        self.excel_uploader = widgets.FileUpload(
            accept='.xlsx,.xls',  # Only accept Excel files
            multiple=False,       # Only one file at a time
            description='Upload Excel File',
            button_style='success'  # Green color
        )
        
        # Text input for specifying which column contains the chemical formulas
        self.column_name_input = widgets.Text(
            value='Composition',
            description='Formula Column:',
            placeholder='Name of column with formulas',
            disabled=False,
            style={'description_width': '140px'},
            tooltip='Specify the exact name of the column containing chemical compositions'
        )
        
        # Button to process the Excel file
        self.process_excel_button = widgets.Button(
            description='Process Excel File',
            button_style='info',  # Orange color
            tooltip='Click to analyze all formulas in the Excel file',
            icon='table',
            layout=widgets.Layout(width='250px', height='40px')
        )
        
        # Status display for Excel processing
        self.excel_status_display = widgets.HTML(value="")
        
        # Area to display Excel processing results
        self.excel_results_display = widgets.HTML(value="")
        
        # ----------------
        # Interactive Plot Section Widgets
        # ----------------
        
        # Formula input for plotting
        self.plot_formula_input = widgets.Text(
            value='NaFeO2',
            description='Formula:',
            placeholder='Enter formula (e.g., NaFeO2)',
            style={'description_width': '100px'}
        )
        
        # Alkali metal dropdown
        self.alkali_metal_dropdown = widgets.Dropdown(
            options=['Li', 'Na', 'K', 'Rb', 'Cs'],
            value='Na',
            description='Alkali Metal:',
            style={'description_width': '100px'}
        )
        
        # Interval slider (log scale)
        self.interval_slider = widgets.FloatLogSlider(
            value=0.01,
            base=10,
            min=-3,
            max=-1,
            step=0.1,
            description='Interval:',
            style={'description_width': '100px'}
        )
        
        # Max range slider
        self.max_range_slider = widgets.FloatSlider(
            value=1.0,
            min=0.1,
            max=2.0,
            step=0.1,
            description='Max Range:',
            style={'description_width': '100px'}
        )
        
        # Button to generate plot
        self.generate_plot_button = widgets.Button(
            description='Generate Plot',
            button_style='success',
            tooltip='Click to generate charge balance plot',
            icon='line-chart',
            layout=widgets.Layout(width='150px', height='35px')
        )
        
        # Status display for plotting
        self.plot_status_display = widgets.HTML(value="Configure parameters and click 'Generate Plot'")
        
        # Output widget for displaying plots
        self.plot_output = widgets.Output()
    
    def _setup_event_handlers(self):
        """Connect button clicks to their handler methods"""
        self.calculate_button.on_click(self.handle_single_formula_calculation)
        self.process_excel_button.on_click(self.handle_excel_processing)
        self.generate_plot_button.on_click(self.handle_plot_generation)
        
        # Auto-update plot when parameters change
        self.plot_formula_input.observe(self.on_plot_param_change, names='value')
        self.alkali_metal_dropdown.observe(self.on_plot_param_change, names='value')
        self.interval_slider.observe(self.on_plot_param_change, names='value')
        self.max_range_slider.observe(self.on_plot_param_change, names='value')
    
    def on_plot_param_change(self, change):
        """Handle parameter changes for auto-update"""
        # You can uncomment the line below for auto-update on parameter change
        self.handle_plot_generation(None)
        pass
    
    def _create_tabbed_layout(self):
        """Create and return the complete tabbed user interface layout"""
        
        # Tab 1: Single Formula Analysis
        single_formula_tab = widgets.VBox([
            widgets.HTML("<h3 style='color: #495057; margin-bottom: 15px;'> Single Formula Analysis</h3>"),
            widgets.HTML("<p style='color: #6c757d; margin-bottom: 20px;'>Enter a chemical formula (like H₂SO₄ or CaCl₂) to calculate the oxidation states of each element.</p>"),
            widgets.HBox([self.formula_input, self.calculate_button], layout=widgets.Layout(align_items='flex-end')),
            self.single_results_display
        ], layout=widgets.Layout(padding='20px'))
        
        # Tab 2: Excel File Processing
        excel_tab = widgets.VBox([
            widgets.HTML("<h3 style='color: #495057; margin-bottom: 15px;'> Excel File Processing</h3>"),
            widgets.HTML("<p style='color: #6c757d; margin-bottom: 20px;'>Upload an Excel file containing chemical compositions to analyze multiple formulas at once.</p>"),
            widgets.HBox([self.excel_uploader, self.column_name_input], layout=widgets.Layout(align_items='flex-end')),
            self.process_excel_button,
            self.excel_status_display,
            self.excel_results_display
        ], layout=widgets.Layout(padding='20px'))
        
        # Tab 3: Interactive Charge Balance Plot
        plot_tab = widgets.VBox([
            widgets.HTML("<h3 style='color: #495057; margin-bottom: 15px;'> Interactive Charge Balance Plot</h3>"),
            widgets.HTML("<p style='color: #6c757d; margin-bottom: 20px;'>Visualize how charge balance changes with varying alkali metal amounts.</p>"),
            widgets.HBox([
                widgets.VBox([self.plot_formula_input, self.alkali_metal_dropdown]),
                widgets.VBox([self.interval_slider, self.max_range_slider]),
                widgets.VBox([self.generate_plot_button], layout=widgets.Layout(justify_content='center'))
            ], layout=widgets.Layout(align_items='flex-start')),
            self.plot_status_display,
            self.plot_output
        ], layout=widgets.Layout(padding='20px'))
        
        # Create the tab widget
        tab_widget = widgets.Tab(children=[single_formula_tab, excel_tab, plot_tab])
        tab_widget.set_title(0, 'Single Formula')
        tab_widget.set_title(1, 'Excel Processing')
        tab_widget.set_title(2, 'Interactive Plot')
        
        # Main interface with header and tabs
        main_interface = widgets.VBox([
            widgets.HTML("""
            <div style='text-align: center; padding: 20px; background: linear-gradient(135deg, #1e3c72 0%, #2a5298 100%); color: white; border-radius: 10px; margin-bottom: 20px; box-shadow: 0 4px 6px rgba(0,0,0,0.1);'>
                <h1 style='margin: 0; font-size: 32px; color: white; font-weight: 300;'>
                    appr<span style="color: #fd7e14; font-weight: bold;">OX</span>imate
                </h1>
                <p style='margin: 10px 0 0 0; font-size: 18px; opacity: 0.9; font-weight: 300;'>
                    Advanced Oxidation State Analysis Suite
                </p>
                <p style='margin: 5px 0 0 0; font-size: 14px; opacity: 0.7;'>
                    Single Analysis • Batch Processing • Interactive Visualization
                </p>
            </div>
            """),
            tab_widget
        ])
        
        return main_interface
    
    # ================================
    # EVENT HANDLER METHODS
    # ================================
    
    def handle_single_formula_calculation(self, button_click):
        """Handle when user clicks the calculate button for single formula"""
        
        user_formula = self.formula_input.value.strip()
        if not user_formula:
            self.single_results_display.value = "Please enter a chemical formula"
            return
        
        # Process the formula
        results_table, summary_info = self.process_single_formula(user_formula)
        
        if results_table is not None:
            # Store results for user access
            self.single_result_df = results_table
            
            # Convert the results table to HTML for display
            html_table = results_table.to_html(
                index=False,  # Don't show row numbers
                escape=False,
                classes='styled-table',
                table_id='single-formula-table'
            )
            
            # Create styled HTML display
            self.single_results_display.value = f'''
            <div style="margin-top: 15px;">
                <h4 style="color: #2c3e50;">Analysis Results</h4>
                <p><strong>Formula Analyzed:</strong> <code style="background-color: #f8f9fa; padding: 2px 6px; border-radius: 3px;">{user_formula}</code></p>
                <p><strong>{summary_info}</strong></p>
                <div style="max-height: 400px; overflow: auto; border: 2px solid #fd7e14; border-radius: 8px; margin: 10px 0;">
                    <style>
                        .styled-table {{
                            width: 100%;
                            border-collapse: collapse;
                            margin: 0;
                            font-family: Arial, sans-serif;
                            font-size: 14px;
                        }}
                        .styled-table th {{
                            background-color: #fd7e14;
                            color: white;
                            padding: 12px 8px;
                            text-align: center;
                            font-weight: bold;
                            position: sticky;
                            top: 0;
                            z-index: 10;
                        }}
                        .styled-table td {{
                            padding: 10px 8px;
                            text-align: center;
                            border-bottom: 1px solid #dee2e6;
                        }}
                        .styled-table tr:nth-child(even) {{
                            background-color: #f8f9fa;
                        }}
                        .styled-table tr:hover {{
                            background-color: #e3f2fd;
                        }}
                    </style>
                    {html_table}
            '''
        else:
            self.single_results_display.value = f'<div style="color: #dc3545; font-weight: bold;"> {summary_info}</div>'
            self.single_result_df = None
    
    def handle_excel_processing(self, button_click):
        """Handle when user clicks the process Excel button"""
        
        # Check if a file was uploaded
        if not self.excel_uploader.value:
            self.excel_status_display.value = '<div style="color: #dc3545;"> Please upload an Excel file first</div>'
            return
        
        try:
            # Get the uploaded file content
            # Handle different formats of uploaded file data
            if isinstance(self.excel_uploader.value, tuple):
                uploaded_file = self.excel_uploader.value[0]
            else:
                uploaded_file = list(self.excel_uploader.value.values())[0]
            
            # Extract file content
            if hasattr(uploaded_file, 'content'):
                file_content = uploaded_file.content
            elif isinstance(uploaded_file, dict):
                file_content = uploaded_file['content']
            else:
                file_content = uploaded_file
            
            self.excel_status_display.value = '<div style="color: #fd7e14;"> Processing Excel file...</div>'
            
            # Process the Excel file
            results_table, status_message = self.process_excel_file(file_content, self.column_name_input.value)
            
            if results_table is not None:
                # Store results for user access
                self.processed_df = results_table
                self.excel_status_display.value = f'<div style="color: #28a745;"> Success! Processed {len(results_table)} rows.</div>'
                
                # Convert results to HTML table
                html_table = results_table.to_html(
                    index=False,
                    escape=False,
                    classes='styled-table',
                    table_id='excel-results-table'
                )
                
                # Display the results
                self.excel_results_display.value = f'''
                <div style="margin-top: 15px;">
                    <h4 style="color: #2c3e50;">Excel Processing Results</h4>
                    <p><strong>Total Rows Processed:</strong> {len(results_table)}</p>
                    <p><strong>Formula Column:</strong> <code style="background-color: #fd7e14; padding: 2px 6px; border-radius: 3px;">{self.column_name_input.value}</code></p>
                    <div style="max-height: 500px; overflow: auto; border: 2px solid #fd7e14; border-radius: 8px; margin: 10px 0;">
                        <style>
                            #excel-results-table {{
                                width: 100%;
                                border-collapse: collapse;
                                margin: 0;
                                font-family: Arial, sans-serif;
                                font-size: 12px;
                            }}
                            #excel-results-table th {{
                                background-color: #fd7e14;
                                color: white;
                                padding: 10px 6px;
                                text-align: center;
                                font-weight: bold;
                                position: sticky;
                                top: 0;
                                z-index: 10;
                            }}
                            #excel-results-table td {{
                                padding: 8px 6px;
                                text-align: center;
                                border-bottom: 1px solid #dee2e6;
                            }}
                            #excel-results-table tr:nth-child(even) {{
                                background-color: #f8f9fa;
                            }}
                            #excel-results-table tr:hover {{
                                background-color: #d4edda;
                            }}
                        </style>
                        {html_table}
                    </div>
                    <p style="margin-top: 10px; font-style: italic; color: #6c757d;">
                        💡 The results are also stored in the variable <code>calculator.processed_df</code> for further analysis.
                    </p>
                </div>
                '''
            else:
                self.excel_status_display.value = f'<div style="color: #dc3545;"> Error: {status_message}</div>'
                self.excel_results_display.value = ""
                
        except Exception as error:
            self.excel_status_display.value = f'<div style="color: #dc3545;"> Error: {str(error)}</div>'
            self.excel_results_display.value = ""
            
    def handle_plot_request(self, formula='NaFeO2', alkali_metal='Na', interval=0.01, max_range=2.0):
        """Handles full workflow from parsing to plotting."""
        try:
            parsed_formula = self.analyzer.parse_formula(formula)
            if not parsed_formula:
                self.plot_status_display.value = '<div style="color: #dc3545;">Error: Could not parse formula</div>'
                return

            values, charge_balances, status = self.compute_charge_balances(parsed_formula, alkali_metal, interval, max_range)
            
            if status != "Success":
                self.plot_status_display.value = f'<div style="color: #dc3545;">{status}</div>'
                return
                
            if len(values) == 0:
                self.plot_status_display.value = '<div style="color: #dc3545;">No data points generated. Check your parameters.</div>'
                return
                
            fig = self.create_charge_balance_plot(values, charge_balances, alkali_metal, formula)
            
            # Clear previous plot and show new one
            with self.plot_output:
                self.plot_output.clear_output(wait=True)
                fig.show()
                
            self.plot_status_display.value = f'<div style="color: #28a745;">Plot generated successfully for {formula}</div>'
            
        except Exception as e:
            self.plot_status_display.value = f'<div style="color: #dc3545;">Error: {str(e)}</div>'
    
    def handle_plot_generation(self, button_click):
        """Handle when user clicks the generate plot button"""
        formula = self.plot_formula_input.value.strip()
        alkali_metal = self.alkali_metal_dropdown.value
        interval = self.interval_slider.value
        max_range = self.max_range_slider.value
        
        if not formula:
            self.plot_status_display.value = '<div style="color: #dc3545;">Please enter a chemical formula</div>'
            return
            
        self.handle_plot_request(formula, alkali_metal, interval, max_range)

    # ================================
    # DATA ACCESS
    # ================================

    def display(self):
        """Display the calculator interface"""
        display(self.interface)
    
    def get_single_results(self):
        """Get the results from single formula processing"""
        return self.single_result_df
    
    def get_excel_results(self):
        """Get the results from Excel file processing"""
        return self.processed_df

# Initialize the analyzer (assumes ApprOXimate class is available)
analyzer = ApprOXimate('all_elements_oxidation_states(in).csv')

# Create the calculator instance
calculator = OxidationStateCalculator(analyzer)

# Display the user interface
calculator.display()